In [27]:
import csv
import re
import spacy
from spacy import displacy
from __future__ import print_function

# pre-processing
def filter(senSet):
    #remove content between [ ]
    print("Pre-processing...")
    for index in range(len(senSet)):
        while senSet[index].find('[')>=0:
            i_start = senSet[index].find('[')
            i_end = senSet[index].find(']')
            s = senSet[index][i_start:i_end+2]
            senSet[index] = senSet[index].replace(s, "")

#load data
file = open("shortdataset.csv", "r")
#file = open("newdataset_formatted.csv", "r")
reader = csv.reader(file)

senSet = []
for item in reader:
    #format sentences in item as string
    fullP = "".join(item)
    splitP = fullP.split(";", 3);
    splitS = splitP[3][1:len(splitP[3])].split(".");
    #print(splitS)
    for sen in splitS:
        senSet.append(sen)#store the sentence into an array

file.close()
print("Total sentences: " + str(len(senSet)))

#pre-processing
filter(senSet)

'''for sentence in senSet:
    print(senSet)'''

#parse sentence
nlp = spacy.load('en_core_web_sm')

#for index in range(len(senSet)):
index = 0
doc = nlp(str(senSet[index]))
print(str(index) + ': ' + senSet[index])

#print the result
sub = ""
pred = ""
obj = ""
stop_index = []

print('stop words: ', end='')
for token in doc:
    #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
    #      token.shape_, token.is_alpha, token.is_stop)
    
    # record the index of stop words
    if token.is_stop:
        print(token.text + ', ', end='')
        stop_index.append(token.i)
    if re.match('nsubj', token.dep_):   
        subj = token.text
    if re.match('ROOT', token.dep_): 
        pred = token.lemma_
        pred_orig = token.text
    if re.match('dobj', token.dep_): 
        obj = token.lemma_
print('\n')

# using chunk to update subject and object
for chunk in doc.noun_chunks:
    if chunk.root.head.text == pred_orig and re.match('nsubj', chunk.root.dep_):
        subj = chunk.text
        # remove stop words
        '''i_stop=0
        for i_sen in range(chunk.start, chunk.end):
            while i_stop < len(stop_index) and stop_index[i_stop] < i_sen-1:
                #print(str(stop_index[i_stop]) + ' ' + str(i_sen))
                i_stop = i_stop+1
            # there is no stop word in current chunk
            if i_stop >= len(stop_index):
                break;
            #print(i_sen)
            # finish going through the chunk
            if stop_index[i_stop] > chunk.end-1:
                break
            # find the stop word and remove it
            if stop_index[i_stop] == i_sen-1:
                #print(doc[i_sen-1])
                if i_sen-1 == chunk.start:
                    subj = subj.replace(doc[i_sen-1].text + ' ', '')
                else:
                    subj = subj.replace(' ' + doc[i_sen-1].text, '')'''

    if chunk.root.head.text == pred_orig and re.match('dobj|attr', chunk.root.dep_):
        obj = chunk.text
        # remove stop words
        '''i_stop=0
        for i_sen in range(chunk.start, chunk.end):
            while i_stop < len(stop_index) and stop_index[i_stop] < i_sen-1:
                #print(str(stop_index[i_stop]) + ' ' + str(i_sen))
                i_stop = i_stop+1
            # there is no stop word in current chunk
            if i_stop >= len(stop_index):
                break;
            #print(i_sen)
            # finish going through the chunk
            if stop_index[i_stop] > chunk.end-1:
                break
            # find the stop word and remove it
            if stop_index[i_stop] == i_sen-1:
                #print(doc[i_sen-1])
                if i_sen-1 == chunk.start:
                    obj = obj.replace(doc[i_sen-1].text + ' ', '')
                else:
                    obj = obj.replace(' ' + doc[i_sen-1].text, '')'''

    #print(chunk.text + ' ' + str(chunk.start))
    #print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

#print(subj + ' - ' + pred + ' - ' + obj + '\n')

# second method to remove stop words
subj_new = ''
doc_subj = nlp(subj)
for token in doc_subj:
    #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
    #      token.shape_, token.is_alpha, token.is_stop)
    if ~token.is_stop:
        subj_new = subj_new + token.text + ' '

obj_new = ''
doc_obj = nlp(obj)
for token in doc_obj:
    # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
    #       token.shape_, token.is_alpha, token.is_stop)
    if not token.is_stop:
        obj_new = obj_new + token.text + ' '
print(subj_new + ' - ' + pred + ' - ' + obj_new + '\n')

## visualize the semantic tree
#options = {'compact': True, 'color': 'blue'}
#displacy.serve(doc, style='dep', options=options)
#displacy.serve(doc, style='dep')

Total sentences: 46
Pre-processing...
0: Sure um start off - I'm a Purdue grad
stop words: off, a, 

I  - be - Purdue grad 

